## 클라우드 모드 테스트

### 스크립트 파일 작성

In [3]:
%%writefile src/train.py
import argparse
import os
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import joblib

def model_fn(model_dir):
    """모델 로드 함수"""
    print("모델 로딩 중...")
    model = joblib.load(os.path.join(model_dir, 'model.joblib'))
    scaler = joblib.load(os.path.join(model_dir, 'scaler.joblib'))
    print("모델 로딩 완료")
    return {'model': model, 'scaler': scaler}

def predict_fn(input_data, model):
    """예측 함수"""
    scaler = model['scaler']
    clf = model['model']
    scaled_input = scaler.transform(input_data)
    predictions = clf.predict(scaled_input)
    return predictions

if __name__ == '__main__':
    print("프로그램 시작")
    
    parser = argparse.ArgumentParser()
    
    # 하이퍼파라미터 등 파라미터 추가
    parser.add_argument('--n-estimators', type=int, default=100)
    parser.add_argument('--min-samples-split', type=int, default=2)
    
    # SageMaker 특정 인자
    parser.add_argument('--model-dir', type=str, default=os.environ.get('SM_MODEL_DIR'))
    
    args, _ = parser.parse_known_args()
    print(f"파라미터 설정: n_estimators={args.n_estimators}, min_samples_split={args.min_samples_split}")
    
    # Iris 데이터셋 로드
    print("Iris 데이터셋 로드 중...")
    iris = load_iris()
    X, y = iris.data, iris.target
    print("데이터셋 로드 완료")
    
    # 데이터 전처리
    print("데이터 전처리 시작")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    print("데이터 전처리 완료")
    
    # 모델 훈련
    print("모델 훈련 시작")
    model = RandomForestClassifier(n_estimators=args.n_estimators, min_samples_split=args.min_samples_split, random_state=2024)
    model.fit(X_train_scaled, y_train)
    print("모델 훈련 완료")
    
    # 모델 평가
    print("모델 평가 중...")
    y_pred = model.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)
    print(f'테스트 정확도: {accuracy}')
    
    if args.model_dir:
        print("모델 및 스케일러 저장 중...")
        # 모델 저장
        joblib.dump(model, os.path.join(args.model_dir, 'model.joblib'))
    
        # 스케일러 저장 (추론 시 사용)
        joblib.dump(scaler, os.path.join(args.model_dir, 'scaler.joblib'))
        print("모델 및 스케일러 저장 완료")
    
    print("프로그램 종료")

Writing src/train.py


### 로컬 테스트

In [4]:
!python src/train.py --n-estimators 100 --min-samples-split 3

프로그램 시작
파라미터 설정: n_estimators=100, min_samples_split=3
Iris 데이터셋 로드 중...
데이터셋 로드 완료
데이터 전처리 시작
데이터 전처리 완료
모델 훈련 시작
모델 훈련 완료
모델 평가 중...
테스트 정확도: 1.0
프로그램 종료


### 클라우드 모드 실행

라이브러리 및 환경변수 준비

In [6]:
import sagemaker
import boto3
import os
from dotenv import load_dotenv
from sagemaker.sklearn.estimator import SKLearn
load_dotenv()

/Users/dante/workspace/dante-code/projects/kmu-cloud-ml-2025/.conda/lib/python3.12/site-packages/sagemaker_core/main/shapes.py:2509: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  class MonitoringDatasetFormat(Base):


[10/11/25 13:59:37] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=922877;file:///Users/dante/workspace/dante-code/projects/kmu-cloud-ml-2025/.conda/lib/python3.12/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=167291;file:///Users/dante/workspace/dante-code/projects/kmu-cloud-ml-2025/.conda/lib/python3.12/site-packages/botocore/credentials.py#1278\1278]8;;\

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/dante/Library/Application Support/sagemaker/config.yaml


True

SDK 준비 : 세션과 IAM 실행역할

In [7]:
boto3_session = boto3.Session(profile_name='awstutor')
sagemaker_session = sagemaker.Session(boto_session=boto3_session)
role = os.environ.get('SAGEMAKER_EXECUTION_ROLE_ARN')

[10/11/25 13:59:43] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=648467;file:///Users/dante/workspace/dante-code/projects/kmu-cloud-ml-2025/.conda/lib/python3.12/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=453213;file:///Users/dante/workspace/dante-code/projects/kmu-cloud-ml-2025/.conda/lib/python3.12/site-packages/botocore/credentials.py#1278\1278]8;;\

모델 훈련 스크립트 및 파라미터 설정

In [8]:
estimator = SKLearn(
    entry_point='train.py',
    source_dir='src',
    role=role,
    sagemaker_session=sagemaker_session,
    instance_count=1,
    instance_type='ml.m5.large',
    framework_version='0.23-1',
    hyperparameters={
        'n-estimators': 150,
        'min-samples-split': 3
    }
)

[10/11/25 14:00:17] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=309155;file:///Users/dante/workspace/dante-code/projects/kmu-cloud-ml-2025/.conda/lib/python3.12/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=427063;file:///Users/dante/workspace/dante-code/projects/kmu-cloud-ml-2025/.conda/lib/python3.12/site-packages/botocore/credentials.py#1278\1278]8;;\

모델 훈련 실행

In [9]:
estimator.fit()

[10/11/25 14:00:27] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=958025;file:///Users/dante/workspace/dante-code/projects/kmu-cloud-ml-2025/.conda/lib/python3.12/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=799305;file:///Users/dante/workspace/dante-code/projects/kmu-cloud-ml-2025/.conda/lib/python3.12/site-packages/sagemaker/telemetry/telemetry_logging.py#90\90]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

[10/11/25 14:00:28] INFO     Creating training-job with name:                                       ]8;id=68441;file:///Users/dante/workspace/dante-code/projects/kmu-cloud-ml-2025/.conda/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=122264;file:///Users/dante/workspace/dante-code/projects/kmu-cloud-ml-2025/.conda/lib/python3.12/site-packages/sagemaker/session.py#1042\1042]8;;\
                             sagemaker-scikit-learn-2025-10-11-05-00-27-554                                        

2025-10-11 05:00:30 Starting - Starting the training job...
2025-10-11 05:00:44 Starting - Preparing the instances for training...
2025-10-11 05:01:34 Downloading - Downloading the training image......
2025-10-11 05:02:35 Training - Training image download completed. Training in progress.
2025-10-11 05:02:35 Uploading - Uploading generated training model2025-10-11 05:02:28,945 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2025-10-11 05:02:28,948 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-10-11 05:02:28,987 sagemaker_sklearn_container.training INFO     Invoking user training script.
2025-10-11 05:02:29,151 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-10-11 05:02:29,163 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-10-11 05:02:29,174 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-

추론(예측) 엔드포인트 배포

In [10]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

[10/11/25 14:27:03] INFO     Creating model with name:                                              ]8;id=758561;file:///Users/dante/workspace/dante-code/projects/kmu-cloud-ml-2025/.conda/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=737491;file:///Users/dante/workspace/dante-code/projects/kmu-cloud-ml-2025/.conda/lib/python3.12/site-packages/sagemaker/session.py#4025\4025]8;;\
                             sagemaker-scikit-learn-2025-10-11-05-27-02-909                                        

                    INFO     Creating endpoint-config with name                                     ]8;id=832622;file:///Users/dante/workspace/dante-code/projects/kmu-cloud-ml-2025/.conda/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=506428;file:///Users/dante/workspace/dante-code/projects/kmu-cloud-ml-2025/.conda/lib/python3.12/site-packages/sagemaker/session.py#5820\5820]8;;\
                             sagemaker-scikit-learn-2025-10-11-05-27-02-909                                        

[10/11/25 14:27:04] INFO     Creating endpoint with name                                            ]8;id=850229;file:///Users/dante/workspace/dante-code/projects/kmu-cloud-ml-2025/.conda/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=370652;file:///Users/dante/workspace/dante-code/projects/kmu-cloud-ml-2025/.conda/lib/python3.12/site-packages/sagemaker/session.py#4642\4642]8;;\
                             sagemaker-scikit-learn-2025-10-11-05-27-02-909                                        

---!

실시간 예측 실행

In [11]:
from sagemaker.serializers import CSVSerializer
predictor.serializer = CSVSerializer()

In [12]:
from sklearn.datasets import load_iris
import numpy as np

# 테스트용 데이터 준비
iris = load_iris()
test_data = iris.data[:5]  # 처음 5개 샘플 사용

# 예측
# predictor.predict() 메서드는 NumPy 배열을 예상하므로 데이터를 변환합니다
result = predictor.predict(CSVSerializer().serialize(test_data))
print("예측결과:", result)

예측결과: [0 0 0 0 0]


엔드포인트 삭제

In [13]:
predictor.delete_endpoint()

[10/11/25 14:51:57] INFO     Deleting endpoint configuration with name:                             ]8;id=295635;file:///Users/dante/workspace/dante-code/projects/kmu-cloud-ml-2025/.conda/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=201828;file:///Users/dante/workspace/dante-code/projects/kmu-cloud-ml-2025/.conda/lib/python3.12/site-packages/sagemaker/session.py#4796\4796]8;;\
                             sagemaker-scikit-learn-2025-10-11-05-27-02-909                                        

                    INFO     Deleting endpoint with name:                                           ]8;id=869062;file:///Users/dante/workspace/dante-code/projects/kmu-cloud-ml-2025/.conda/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=415963;file:///Users/dante/workspace/dante-code/projects/kmu-cloud-ml-2025/.conda/lib/python3.12/site-packages/sagemaker/session.py#4786\4786]8;;\
                             sagemaker-scikit-learn-2025-10-11-05-27-02-909                                        